# Take Control of GPT : Use the API. Don't let the API use you

### My first openai API Project!!!

Here, I use the openai Python lib and some custom classes to handle processing prompts and responses. I store the results in a pandas dataframe, then format the code in a pandas 'style' object. I can use the dataframe and other objects (like the multiline string of prompts) to manage context for later prompts. Yes, context from prompts and answers. So much more to do with this. But this is a start.

## Setup

In [1]:
%%html
<style>
    /* Enable word wrap */
    .rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
      white-space: nowrap;
      overflow: hidden;
      text-overflow: ellipsis;
    }
</style>

### Imports

In [2]:
# ...
import openai
import os
import pprint
import sys
from pprint import pprint
import json
import pyarrow as pa
import pandas as pd

lib = os.path.abspath(os.path.join('..', '..', 'libs'))
sys.path.append(lib)

from MessageParser import MessageParser
from ResponseParser import ResponseParser
from PromptLogs import PromptLogs


### Config

In [3]:
# ...
# *******************************************************************************************************
# SET THE API KEY
# ================
# -- set the env variable OPENAI_API_KEY to your OpenAI API key       <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
openai.api_key = os.getenv("OPENAI_API_KEY")
# *******************************************************************************************************

# Setup up pandas properties
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line break in DataFrame
pd.set_option('display.max_colwidth', None)  # Display the full content of cells



## Main code

Prep the system prompts

In [4]:
sys_prompt_list = []
sys_prompt_1 = """You are a software expert."""
sys_prompt_2 = """Use. "```python" style for code blocks."""
sys_prompt_3 = """Include the name for a file where I would store individual code for functions or classes."""

sys_prompt_list.append(sys_prompt_1)
sys_prompt_list.append(sys_prompt_2)
sys_prompt_list.append(sys_prompt_3)

sys_prompt_string = "".join(sys_prompt_list)

### a PromptLogs object will hold all the individual individual parser objects, which hold the response from a single exchange

In [5]:
logs = PromptLogs()

Generate the message and make the call

In [6]:
message_list = f"""
system: {sys_prompt_string}
user: Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.
"""

# a MessageParser object will prcoess the message_list and make an acceptable messages object
response_parser = MessageParser(message_list)
messages = response_parser.parse_messages()

#  make the openai call and get the response
# I'll abstract this away in the future
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)


a response parser will process the response

In [7]:
response_parser = ResponseParser(response=response)

get a pandas dataframe of the logged prompts and responses

In [8]:
response_df = response_parser.get_df()
response_df

,id,created,model,usage,all_code,code_blocks,content
0,chatcmpl-73yiHMFVb18aDGPQ78lsuzWm4lmwx,1.681183e+09,gpt-3.5-turbo-0301,66.0,"def&nbsp;fibonacci_sequence(n):<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;This&nbsp;function&nbsp;generates&nbsp;the&nbsp;fibonacci&nbsp;sequence&nbsp;of&nbsp;n&nbsp;items.<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;sequence&nbsp;=&nbsp;[0,&nbsp;1]<br>&nbsp;&nbsp;&nbsp;&nbsp;while&nbsp;len(sequence)&nbsp;<&nbsp;n:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;next_item&nbsp;=&nbsp;sequence[-1]&nbsp;+&nbsp;sequence[-2]<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;sequence.append(next_item)<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;sequence<br>def&nbsp;pad_string(string):<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;This&nbsp;function&nbsp;pads&nbsp;a&nbsp;given&nbsp;string&nbsp;with&nbsp;zeroes&nbsp;on&nbsp;the&nbsp;left&nbsp;so&nbsp;that&nbsp;it<br>&nbsp;&nbsp;&nbsp;&nbsp;is&nbsp;10&nbsp;characters&nbsp;long.<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;string.zfill(10)<br>","[\ndef fibonacci_sequence(n):\n """"""\n This function generates the fibonacci sequence of n items.\n """"""\n sequence = [0, 1]\n while len(sequence) < n:\n next_item = sequence[-1] + sequence[-2]\n sequence.append(next_item)\n return sequence\n, \ndef pad_string(string):\n """"""\n This function pads a given string with zeroes on the left so that it\n is 10 characters long.\n """"""\n return string.zfill(10)\n]","Code to generate the fibonacci sequence of n numbers:\n\n```python\ndef fibonacci_sequence(n):\n """"""\n This function generates the fibonacci sequence of n items.\n """"""\n sequence = [0, 1]\n while len(sequence) < n:\n next_item = sequence[-1] + sequence[-2]\n sequence.append(next_item)\n return sequence\n```\n\nHere is a function to zero pad a string to 10 characters:\n\n```python\ndef pad_string(string):\n """"""\n This function pads a given string with zeroes on the left so that it\n is 10 characters long.\n """"""\n return string.zfill(10)\n```"


In [9]:
# A pandas style applies custom formatting to a DataFrame to make it easier to read and
# understand. It does not offer the full range of functionality that a DataFrame provides.
response_parser.styled_log

,id,created,model,usage,all_code
0,chatcmpl-73yiHMFVb18aDGPQ78lsuzWm4lmwx,1681182721.000000,gpt-3.5-turbo-0301,66.000000,"def fibonacci_sequence(n): """""" This function generates the fibonacci sequence of n items. """""" sequence = [0, 1] while len(sequence) < n: next_item = sequence[-1] + sequence[-2] sequence.append(next_item) return sequencedef pad_string(string): """""" This function pads a given string with zeroes on the left so that it is 10 characters long. """""" return string.zfill(10)"


In [10]:
# this is a PromptLogs object again. We'll add the response_parser to the logs
logs.append(response_parser)

## And we have another exchange

In [11]:
message_list = f"""
system: {sys_prompt_string}
user: Write a python function that returns numbers 1-10 in a list. 
"""

message_parser = MessageParser(message_list)
messages = message_parser.parse_messages()
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)
response_parser = ResponseParser(response=response)

logs.append(response_parser)

In [12]:
logs.df

,id,created,model,usage,all_code,code_blocks,content
0,chatcmpl-73yiHMFVb18aDGPQ78lsuzWm4lmwx,1.681183e+09,gpt-3.5-turbo-0301,66.0,"def&nbsp;fibonacci_sequence(n):<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;This&nbsp;function&nbsp;generates&nbsp;the&nbsp;fibonacci&nbsp;sequence&nbsp;of&nbsp;n&nbsp;items.<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;sequence&nbsp;=&nbsp;[0,&nbsp;1]<br>&nbsp;&nbsp;&nbsp;&nbsp;while&nbsp;len(sequence)&nbsp;<&nbsp;n:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;next_item&nbsp;=&nbsp;sequence[-1]&nbsp;+&nbsp;sequence[-2]<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;sequence.append(next_item)<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;sequence<br>def&nbsp;pad_string(string):<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;This&nbsp;function&nbsp;pads&nbsp;a&nbsp;given&nbsp;string&nbsp;with&nbsp;zeroes&nbsp;on&nbsp;the&nbsp;left&nbsp;so&nbsp;that&nbsp;it<br>&nbsp;&nbsp;&nbsp;&nbsp;is&nbsp;10&nbsp;characters&nbsp;long.<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;string.zfill(10)<br>","[\ndef fibonacci_sequence(n):\n """"""\n This function generates the fibonacci sequence of n items.\n """"""\n sequence = [0, 1]\n while len(sequence) < n:\n next_item = sequence[-1] + sequence[-2]\n sequence.append(next_item)\n return sequence\n, \ndef pad_string(string):\n """"""\n This function pads a given string with zeroes on the left so that it\n is 10 characters long.\n """"""\n return string.zfill(10)\n]","Code to generate the fibonacci sequence of n numbers:\n\n```python\ndef fibonacci_sequence(n):\n """"""\n This function generates the fibonacci sequence of n items.\n """"""\n sequence = [0, 1]\n while len(sequence) < n:\n next_item = sequence[-1] + sequence[-2]\n sequence.append(next_item)\n return sequence\n```\n\nHere is a function to zero pad a string to 10 characters:\n\n```python\ndef pad_string(string):\n """"""\n This function pads a given string with zeroes on the left so that it\n is 10 characters long.\n """"""\n return string.zfill(10)\n```"
0,chatcmpl-73yiO6cLGclJXwAicuXt30fT46M14,1.681183e+09,gpt-3.5-turbo-0301,60.0,"def&nbsp;number_list():<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;[i&nbsp;for&nbsp;i&nbsp;in&nbsp;range(1,&nbsp;11)]<br>","[\ndef number_list():\n return [i for i in range(1, 11)]\n]","This function would be saved in a separate .py file, ideally with a name that reflects its purpose. For example, `number_list.py`.\n\n```python\ndef number_list():\n return [i for i in range(1, 11)]\n```\n\nThis function uses a list comprehension to generate and return a list of the numbers 1 through 10 when called."


## And print the styled logs with the code from two separate prompt exchanges

In [14]:
logs.styled_log

,created,model,usage,all_code
id,,,,
chatcmpl-73yiHMFVb18aDGPQ78lsuzWm4lmwx,1681182721.000000,gpt-3.5-turbo-0301,66.000000,"def fibonacci_sequence(n): """""" This function generates the fibonacci sequence of n items. """""" sequence = [0, 1] while len(sequence) < n: next_item = sequence[-1] + sequence[-2] sequence.append(next_item) return sequencedef pad_string(string): """""" This function pads a given string with zeroes on the left so that it is 10 characters long. """""" return string.zfill(10)"
chatcmpl-73yiO6cLGclJXwAicuXt30fT46M14,1681182728.000000,gpt-3.5-turbo-0301,60.000000,"def number_list(): return [i for i in range(1, 11)]"
